In [1]:
import os
import numpy as np
import torch
from PIL import Image
import pickle 
import cv2

class PcbDataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root))))
        self.masks = list(sorted(os.listdir(os.path.join(root))))
#         self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # load images ad masks
        file_path = os.path.join(self.root, self.imgs[idx])
#         mask_path = os.path.join(self.root, "Masks", self.masks[idx])
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        data_dict = np.load(file_path,allow_pickle=True)
        img = data_dict['image']
        #img = np.concatenate([img[0],img[1],img[2]],axis=2)
        img = img[1]
        img = img*255
        img = img.astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        boxes = data_dict['boxes']
#         img = Image.open(img_path).convert("RGB")
        

        # get bounding box coordinates for each mask
        num_objs = len(boxes)

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] ) * (boxes[:, 2])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
#         target["masks"] = []

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [5]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 1 class (PCB) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)

In [6]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# FasterRCNN needs to know the number of
# output channels in a backbone. For mobilenet_v2, it's 1280
# so we need to add it here
backbone.out_channels = 1280

# let's make the RPN generate 5 x 3 anchors per spatial
# location, with 5 different sizes and 3 different aspect
# ratios. We have a Tuple[Tuple[int]] because each feature
# map could potentially have different sizes and
# aspect ratios
anchor_generator = AnchorGenerator(sizes=((16,32, 64, 128),),aspect_ratios=((0.5, 1.0, 2.0),))

# let's define what are the feature maps that we will
# use to perform the region of interest cropping, as well as
# the size of the crop after rescaling.
# if your backbone returns a Tensor, featmap_names is expected to
# be [0]. More generally, the backbone should return an
# OrderedDict[Tensor], and in featmap_names you can choose which
# feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
                                                output_size=7,
                                                sampling_ratio=2)

# put the pieces together inside a FasterRCNN model
model = FasterRCNN(backbone,
                   num_classes=2,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)

IndexError: list index out of range

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

#     # now get the number of input features for the mask classifier
#     in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
#     hidden_layer = 256
#     # and replace the mask predictor with a new one
#     model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
#                                                        hidden_layer,
#                                                        num_classes)

    return model


In [4]:
import transforms as T
import detection_utils

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [28]:
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# dataset = PcbDataset('../PcbData', get_transform(train=True))
# data_loader = torch.utils.data.DataLoader(
#  dataset, batch_size=2, shuffle=True, num_workers=4,
#  collate_fn=detection_utils.collate_fn)
# # For Training
# images,targets = next(iter(data_loader))
# images = list(image for image in images)
# targets = [{k: v for k, v in t.items()} for t in targets]
# output = model(images,targets)   # Returns losses and detections
# # For inference
# model.eval()
# x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
# predictions = model(x)           # Returns predictions

In [7]:
from engine import train_one_epoch, evaluate
import detection_utils


def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print(device)

    # our dataset has two classes only - background and person
    num_classes = 2
    # use our dataset and defined transformations
    dataset = PcbDataset('../PcbData', get_transform(train=True))
    dataset_test = PcbDataset('../PcbData', get_transform(train=False))

    # split the dataset in train and test set
    indices = torch.randperm(len(dataset)).tolist()
    dataset = torch.utils.data.Subset(dataset, indices[:-15])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-15:])

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=4,
        collate_fn=detection_utils.collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=detection_utils.collate_fn)

    # get the model using our helper function
 
    # load a model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    # replace the classifier with a new one, that has
    # num_classes which is user-defined
    num_classes = 2  # 1 class (PCB) + background
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    #     model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 10
#     img,target = dataset.__getitem__(2)
#     print(img.shape)
#     print(target['boxes'].shape)
#     print(target['labels'].shape)
#     print(target['iscrowd'].shape)
#     print(target['area'].shape)
#     print(target['image_id'].shape)
    
    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

main()

cuda
Epoch: [0]  [ 0/93]  eta: 0:01:31  loss_objectness: 6.8169 (6.8169)  loss_rpn_box_reg: 0.5386 (0.5386)  loss_classifier: 0.8101 (0.8101)  loss: 8.6103 (8.6103)  lr: 0.000059  loss_box_reg: 0.4447 (0.4447)  time: 0.9888  data: 0.4550  max mem: 1623
Epoch: [0]  [10/93]  eta: 0:00:26  loss_objectness: 3.3622 (3.5362)  loss_rpn_box_reg: 0.5641 (0.5869)  loss_classifier: 0.6783 (0.6822)  loss: 5.0417 (5.1989)  lr: 0.000602  loss_box_reg: 0.4018 (0.3937)  time: 0.3134  data: 0.0490  max mem: 2101
Epoch: [0]  [20/93]  eta: 0:00:20  loss_objectness: 0.7607 (2.1608)  loss_rpn_box_reg: 0.5170 (0.5507)  loss_classifier: 0.5663 (0.6150)  loss: 2.2642 (3.7174)  lr: 0.001145  loss_box_reg: 0.3913 (0.3911)  time: 0.2428  data: 0.0064  max mem: 2101
Epoch: [0]  [30/93]  eta: 0:00:16  loss_objectness: 0.4382 (1.5860)  loss_rpn_box_reg: 0.4674 (0.5254)  loss_classifier: 0.4848 (0.5520)  loss: 1.7910 (3.0347)  lr: 0.001688  loss_box_reg: 0.3696 (0.3713)  time: 0.2392  data: 0.0044  max mem: 2101
Epo

Epoch: [2]  [ 0/93]  eta: 0:01:02  loss_objectness: 0.0918 (0.0918)  loss_rpn_box_reg: 0.4165 (0.4165)  loss_classifier: 0.2952 (0.2952)  loss: 1.0600 (1.0600)  lr: 0.005000  loss_box_reg: 0.2564 (0.2564)  time: 0.6677  data: 0.4191  max mem: 2107
Epoch: [2]  [10/93]  eta: 0:00:23  loss_objectness: 0.1381 (0.1210)  loss_rpn_box_reg: 0.3015 (0.3341)  loss_classifier: 0.3042 (0.2956)  loss: 0.9841 (0.9710)  lr: 0.005000  loss_box_reg: 0.2302 (0.2203)  time: 0.2782  data: 0.0407  max mem: 2107
Epoch: [2]  [20/93]  eta: 0:00:19  loss_objectness: 0.1381 (0.1261)  loss_rpn_box_reg: 0.3015 (0.3330)  loss_classifier: 0.2885 (0.2820)  loss: 0.9668 (0.9558)  lr: 0.005000  loss_box_reg: 0.2152 (0.2148)  time: 0.2415  data: 0.0037  max mem: 2112
Epoch: [2]  [30/93]  eta: 0:00:16  loss_objectness: 0.1136 (0.1234)  loss_rpn_box_reg: 0.3016 (0.3198)  loss_classifier: 0.2748 (0.2788)  loss: 0.9248 (0.9408)  lr: 0.005000  loss_box_reg: 0.2167 (0.2188)  time: 0.2424  data: 0.0045  max mem: 2112
Epoch: [

Epoch: [4]  [ 0/93]  eta: 0:01:00  loss_objectness: 0.0687 (0.0687)  loss_rpn_box_reg: 0.2488 (0.2488)  loss_classifier: 0.2468 (0.2468)  loss: 0.7543 (0.7543)  lr: 0.000500  loss_box_reg: 0.1900 (0.1900)  time: 0.6521  data: 0.3928  max mem: 2125
Epoch: [4]  [10/93]  eta: 0:00:23  loss_objectness: 0.0707 (0.0747)  loss_rpn_box_reg: 0.2443 (0.2362)  loss_classifier: 0.2384 (0.2318)  loss: 0.7543 (0.7358)  lr: 0.000500  loss_box_reg: 0.1900 (0.1932)  time: 0.2788  data: 0.0392  max mem: 2125
Epoch: [4]  [20/93]  eta: 0:00:19  loss_objectness: 0.0700 (0.0720)  loss_rpn_box_reg: 0.2443 (0.2523)  loss_classifier: 0.2367 (0.2347)  loss: 0.7568 (0.7480)  lr: 0.000500  loss_box_reg: 0.1856 (0.1890)  time: 0.2415  data: 0.0041  max mem: 2125
Epoch: [4]  [30/93]  eta: 0:00:16  loss_objectness: 0.0709 (0.0782)  loss_rpn_box_reg: 0.2779 (0.2650)  loss_classifier: 0.2385 (0.2381)  loss: 0.7568 (0.7726)  lr: 0.000500  loss_box_reg: 0.1856 (0.1913)  time: 0.2413  data: 0.0043  max mem: 2125
Epoch: [

Epoch: [6]  [ 0/93]  eta: 0:01:03  loss_objectness: 0.0194 (0.0194)  loss_rpn_box_reg: 0.1622 (0.1622)  loss_classifier: 0.1485 (0.1485)  loss: 0.4087 (0.4087)  lr: 0.000050  loss_box_reg: 0.0786 (0.0786)  time: 0.6869  data: 0.4434  max mem: 2125
Epoch: [6]  [10/93]  eta: 0:00:23  loss_objectness: 0.0570 (0.0572)  loss_rpn_box_reg: 0.1915 (0.2220)  loss_classifier: 0.2166 (0.2105)  loss: 0.6608 (0.6642)  lr: 0.000050  loss_box_reg: 0.1727 (0.1745)  time: 0.2805  data: 0.0427  max mem: 2125
Epoch: [6]  [20/93]  eta: 0:00:19  loss_objectness: 0.0617 (0.0689)  loss_rpn_box_reg: 0.2453 (0.2254)  loss_classifier: 0.2301 (0.2167)  loss: 0.6831 (0.6932)  lr: 0.000050  loss_box_reg: 0.1758 (0.1823)  time: 0.2413  data: 0.0036  max mem: 2125
Epoch: [6]  [30/93]  eta: 0:00:16  loss_objectness: 0.0660 (0.0707)  loss_rpn_box_reg: 0.2384 (0.2289)  loss_classifier: 0.2347 (0.2215)  loss: 0.7472 (0.7054)  lr: 0.000050  loss_box_reg: 0.1939 (0.1844)  time: 0.2439  data: 0.0046  max mem: 2125
Epoch: [

Epoch: [8]  [ 0/93]  eta: 0:01:00  loss_objectness: 0.1036 (0.1036)  loss_rpn_box_reg: 0.2301 (0.2301)  loss_classifier: 0.2307 (0.2307)  loss: 0.7556 (0.7556)  lr: 0.000050  loss_box_reg: 0.1912 (0.1912)  time: 0.6552  data: 0.4065  max mem: 2125
Epoch: [8]  [10/93]  eta: 0:00:23  loss_objectness: 0.0678 (0.0721)  loss_rpn_box_reg: 0.2485 (0.2594)  loss_classifier: 0.2307 (0.2388)  loss: 0.7556 (0.7610)  lr: 0.000050  loss_box_reg: 0.1891 (0.1906)  time: 0.2805  data: 0.0402  max mem: 2125
Epoch: [8]  [20/93]  eta: 0:00:19  loss_objectness: 0.0629 (0.0695)  loss_rpn_box_reg: 0.2281 (0.2480)  loss_classifier: 0.2429 (0.2385)  loss: 0.7227 (0.7413)  lr: 0.000050  loss_box_reg: 0.1847 (0.1853)  time: 0.2423  data: 0.0039  max mem: 2125
Epoch: [8]  [30/93]  eta: 0:00:16  loss_objectness: 0.0631 (0.0686)  loss_rpn_box_reg: 0.2010 (0.2353)  loss_classifier: 0.2321 (0.2313)  loss: 0.6973 (0.7157)  lr: 0.000050  loss_box_reg: 0.1762 (0.1805)  time: 0.2427  data: 0.0044  max mem: 2125
Epoch: [